In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import pandas_datareader.data as web

In [33]:
sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
sp500.columns = sp500.iloc[0] 
sp500.drop(sp500.index[0], inplace=True)
#sp500['Ticker symbol'][:200].tolist()

symbols_list = ['AAPL', 'AMZN', 'TSLA']

symbols = []

# Iterate through tickers given
for ticker in symbols_list:
    
    start = datetime.datetime(2016, 1, 1)
    end = datetime.datetime(2016, 12, 31)
    
    f = web.DataReader(ticker, 'google', start, end)
    f['Symbol'] = ticker
    symbols.append(f)
    
# Attach Symbols
stocks_data = pd.concat(symbols)

# Add and rename columns
stocks_data= stocks_data[['Symbol','Open','High','Low','Close','Volume']]
stocks_data['Daily Return'] = stocks_data['Close'].pct_change(-1)
stocks_data.reset_index(inplace=True)
#stocks_data.head()

In [90]:
stocks_data.head()

,Date,Symbol,Open,High,Low,Close,Volume,Daily Return
0,2016-01-04,AAPL,102.61,105.37,102.00,105.35,67281190,0.025703
1,2016-01-05,AAPL,105.75,105.85,102.41,102.71,55790992,0.019960
2,2016-01-06,AAPL,100.56,102.37,99.87,100.70,68457388,0.044064
3,2016-01-07,AAPL,98.68,100.13,96.43,96.45,81094428,-0.005260
4,2016-01-08,AAPL,98.55,99.11,96.76,96.96,70798016,-0.015934


In [89]:
# year start
y_start = stocks_data.groupby('Date').get_group('2016-01-04')[['Open']]
y_start['Symbol'] = stocks_data['Symbol']
y_start

# year end
y_end = stocks_data.groupby('Date').get_group('2016-12-30')[['Close']]
y_end['Symbol'] = stocks_data['Symbol']

# merge columns
start_end_y = pd.merge(y_start,y_end, on='Symbol')

# rename columns
start_end_y = start_end_y.rename(columns = {"Open": 'Year_Start_Price', "Close": 'Year_End_Price'})


# Function to define  % change on stock value
def percent_change(end_year, start_year):
    return ((end_year - start_year)/start_year)*100

# Call year change function
year_change = percent_change(start_end_y['Year_End_Price'], start_end_y['Year_Start_Price'])

# Build columns for change in year value
start_end_y['Year_%_Change'] = year_change

# Rounding decimals
start_end_y.round(decimals=2)

# Max and Min price during year
stocks_features = stocks_data.groupby('Symbol').agg({'High':'max','Low':'min'}).\
                  rename(columns={'High':'Max_Yearly_Price','Low':'Min_Yearly_Price'})

# Index reset to column
stocks_features.reset_index(inplace=True)

# Merge dataframes
ndf = pd.merge(stocks_features, start_end_y, on='Symbol')
ndf

,Symbol,Min_Yearly_Price,Max_Yearly_Price,Year_Start_Price,Year_End_Price,Year_%_Change
0,AAPL,89.47,118.69,102.61,115.82,12.873989
1,AMZN,474.00,847.21,656.29,749.87,14.258940
2,TSLA,141.05,269.34,230.72,213.69,-7.381241


In [ ]:
def sharpe_ratio(year_change, stock_close):
    return (year_change -1)/np.std(stock_close)

In [ ]:
sharpe_ratio(start_end_y['Year_%_Change'], )